In [1]:
#from parallelrank import findRank
#import numpy as np
#import tensorflow as tf

#import matplotlib.pyplot as plt
#from timeit import default_timer as timer
#from time import sleep

#assert len(tf.config.list_physical_devices('GPU')) > 0

In [1]:
from scipy.sparse import rand
import numpy as np

matrix = rand(10, 100, density=0.2, format='csr', dtype=np.int8)
matrix.data[:] = 1

In [2]:
from parallelrank import find_rank

find_rank(matrix.indices, matrix.indptr, matrix.shape[0], matrix.shape[1])